In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy.stats import norm
from sklearn.preprocessing import StandardScaler
from scipy import stats
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [2]:
base_path = './lianjia_data/'


In [3]:
# 试着读取一个csv文件
baoshandahuaDF = pd.read_csv(base_path+'baoshan_dahua.csv', header=None,
                             error_bad_lines=False, warn_bad_lines=True)

b'Skipping line 90: expected 7 fields, saw 10\n'


In [4]:
baoshandahuaDF.head(3)

,0,1,2,3,4,5,6
0,20200208,宝山,大华,近小区花园位置 满五唯一 户型正气必看好房,460万,2室2厅 | 88.38平米 | 南 北 | 精装 | 中楼层(共6层) | 2002年建 ...,https://image1.ljcdn.com/310000-inspection/pro...
1,20200208,宝山,大华,2002年电梯两房，户型好，采光好，近地铁，产权清晰必看好房,535万,2室2厅 | 95.11平米 | 南 | 简装 | 低楼层(共11层) | 2004年建 | 板楼,https://image1.ljcdn.com/310000-inspection/pro...
2,20200208,宝山,大华,文华苑四期三房，顶楼复式，单价四万四必看好房,499万,3室2厅 | 113.34平米 | 南 北 | 毛坯 | 高楼层(共6层) | 1999年建...,https://image1.ljcdn.com/310000-inspection/pc1...


In [5]:
# 存储全部数据
data = pd.DataFrame()

In [8]:
# 读取所有数据并合并为一个表格
import os
files = os.listdir(base_path)
for file in files:
    file_path = os.path.join(base_path, file)
    if os.path.isfile(file_path) and os.stat(file_path).st_size != 0:
        tmp_df = pd.read_csv(os.path.join(base_path, file), header=None, 
                             error_bad_lines=False, warn_bad_lines=False)
        dist_name = file.split('_')[0]
        town_name = file.split('_')[1][:-4]
        tmp_df.iloc[:, 1] = dist_name
        tmp_df.iloc[:, 2] = town_name
        data = pd.concat([data, tmp_df], axis=0).reset_index(drop=True)
        
    

In [9]:
data.shape

(16206, 8)

In [10]:
# 比最开始的试读数据多一列
data.head(3)

,0,1,2,3,4,5,6,7
0,20200208,xuhui,kangjian,中间楼层，欧式装修，全明边套净收眼底必看好房,375万,2室0厅 | 53.92平米 | 南 | 精装 | 中楼层(共4层) | 1992年建 | 板楼,https://image1.ljcdn.com/310000-inspection/pc1...,NaN
1,20200208,xuhui,kangjian,花苑村紫竹园 2室1厅 468万必看好房,468万,2室1厅 | 67.11平米 | 南 | 简装 | 6层 | 1990年建 | 板楼,https://image1.ljcdn.com/310000-inspection/pc1...,NaN
2,20200208,xuhui,kangjian,南北两房全明户型采光充足 近上海南站地铁站 诚售。必看好房,468万,2室1厅 | 68.03平米 | 南 | 精装 | 低楼层(共6层) | 1991年建 | 板楼,https://image1.ljcdn.com/310000-inspection/pc1...,NaN


In [11]:
# 看看最后一列有多少数据
data.iloc[:, 7].isnull().sum()

16198

In [12]:
# 最后一列只有几个数据，丢弃, 倒数第二列是图片url，也丢弃
data.drop([6, 7], axis=1, inplace=True)

In [13]:
data.head(3)

,0,1,2,3,4,5
0,20200208,xuhui,kangjian,中间楼层，欧式装修，全明边套净收眼底必看好房,375万,2室0厅 | 53.92平米 | 南 | 精装 | 中楼层(共4层) | 1992年建 | 板楼
1,20200208,xuhui,kangjian,花苑村紫竹园 2室1厅 468万必看好房,468万,2室1厅 | 67.11平米 | 南 | 简装 | 6层 | 1990年建 | 板楼
2,20200208,xuhui,kangjian,南北两房全明户型采光充足 近上海南站地铁站 诚售。必看好房,468万,2室1厅 | 68.03平米 | 南 | 精装 | 低楼层(共6层) | 1991年建 | 板楼


In [14]:
columns = ['date', 'district', 'town_name', 'description', 'listed_price', 'attributes']
data.columns = columns

In [15]:
data.head(3)

,date,district,town_name,description,listed_price,attributes
0,20200208,xuhui,kangjian,中间楼层，欧式装修，全明边套净收眼底必看好房,375万,2室0厅 | 53.92平米 | 南 | 精装 | 中楼层(共4层) | 1992年建 | 板楼
1,20200208,xuhui,kangjian,花苑村紫竹园 2室1厅 468万必看好房,468万,2室1厅 | 67.11平米 | 南 | 简装 | 6层 | 1990年建 | 板楼
2,20200208,xuhui,kangjian,南北两房全明户型采光充足 近上海南站地铁站 诚售。必看好房,468万,2室1厅 | 68.03平米 | 南 | 精装 | 低楼层(共6层) | 1991年建 | 板楼


In [16]:
# 分离attributes中各部分
#lst = ['layout', 'total_area', 'direction', 'decoration_rate', 'stairs', 'built_year', 'building_type' ]
attributes = data['attributes'].apply(lambda x: x.split('|', 6))

In [17]:
attributes.shape

(16206,)

In [18]:
attributes.head(3)

0    [2室0厅 ,  53.92平米 ,  南 ,  精装 ,  中楼层(共4层) ,  199...
1    [2室1厅 ,  67.11平米 ,  南 ,  简装 ,  6层 ,  1990年建 , ...
2    [2室1厅 ,  68.03平米 ,  南 ,  精装 ,  低楼层(共6层) ,  199...
Name: attributes, dtype: object

In [19]:
# 看attributes中是否都为相同长度
(attributes.apply(lambda x: len(x)) == 7).all()

False

In [20]:
(attributes.apply(lambda x: len(x)) != 7).sum() / len(attributes)

0.03585091941256325

In [21]:
# 试看某个长度不为7 的属性
for lst in attributes:
    if len(lst) != 7:
        print(lst)
        break

['2室1厅 ', ' 93.34平米 ', ' 南 ', ' 精装 ', ' 28层  ', ' 塔楼']


In [22]:
attributes.apply(lambda x: len(x)).min()

1

In [23]:
for lst in attributes:
    if len(lst) == 1:
        print(lst)
        break

['230万']


In [24]:
# 看一下这个奇怪的数据
data.loc[attributes.apply(lambda x: len(x))==1, :].head()

,date,district,town_name,description,listed_price,attributes
6006,20200208,minhang,pujiang1,浦江丽都，两房电梯,房型方正，南北通户型必看好房,230万
6112,20200208,minhang,pujiang1,楼层高采光好、看房方便提前约、出入方便,配套齐全,190万
6125,20200208,minhang,pujiang1,浦江小丽都，两房电梯,房型方正，南北通透,243万
6128,20200208,minhang,pujiang1,楼层高采光好、近地铁 出入方便,配套齐全。,188万
6133,20200208,minhang,pujiang1,前滩4公里,带100平露台，南北通，采光通风好 急售,855万


In [25]:
# 读入数据时似乎出问题了，看看有哪些区的数据有问题
data.loc[attributes.apply(lambda x: len(x))==1, 'district'].unique()

array(['minhang', 'xuhui'], dtype=object)

In [26]:
data.loc[attributes.apply(lambda x: len(x))!=7, 'district'].unique()

array(['xuhui', 'pudong', 'minhang', 'jiading', 'baoshan'], dtype=object)

In [27]:
data.loc[attributes.apply(lambda x: len(x))!=7, ['district', 'town_name']].apply(lambda x: x.nunique())

district      5
town_name    54
dtype: int64

In [28]:
len(files)

218

In [29]:
data_broken = data.loc[attributes.apply(lambda x: len(x))!=7, ['district', 'town_name']]

In [30]:
data_broken.shape

(581, 2)

In [31]:
data_broken.head()

,district,town_name
9,xuhui,kangjian
250,pudong,nanmatou
326,pudong,nanmatou
329,pudong,nanmatou
440,pudong,nanmatou


In [32]:
data_broken.shape[0] / data.shape[0]

0.03585091941256325

In [33]:
xuhui_kangjian = pd.read_csv(base_path+'xuhui_kangjian.csv',header=None, 
                             error_bad_lines=False, warn_bad_lines=False)
pudong_nanmatou = pd.read_csv(base_path+'pudong_nanmatou.csv',header=None, 
                             error_bad_lines=False, warn_bad_lines=False)

In [34]:
xuhui_kangjian.head()

,0,1,2,3,4,5,6
0,20200208,徐汇,康健,中间楼层，欧式装修，全明边套净收眼底必看好房,375万,2室0厅 | 53.92平米 | 南 | 精装 | 中楼层(共4层) | 1992年建 | 板楼,https://image1.ljcdn.com/310000-inspection/pc1...
1,20200208,徐汇,康健,花苑村紫竹园 2室1厅 468万必看好房,468万,2室1厅 | 67.11平米 | 南 | 简装 | 6层 | 1990年建 | 板楼,https://image1.ljcdn.com/310000-inspection/pc1...
2,20200208,徐汇,康健,南北两房全明户型采光充足 近上海南站地铁站 诚售。必看好房,468万,2室1厅 | 68.03平米 | 南 | 精装 | 低楼层(共6层) | 1991年建 | 板楼,https://image1.ljcdn.com/310000-inspection/pc1...
3,20200208,徐汇,康健,桂莘小区 双南双天井 精装三房 近地铁十二号线必看好房,520万,3室1厅 | 78.38平米 | 南 | 精装 | 低楼层(共6层) | 1997年建 | 板楼,https://image1.ljcdn.com/310000-inspection/pc1...
4,20200208,徐汇,康健,精装电梯三房，近地铁，双阳台，闹中取静，值得拥有。必看好房,499万,3室1厅 | 110.66平米 | 东南 | 精装 | 28层 | 1996年建 | 塔楼,https://image1.ljcdn.com/310000-inspection/188...


In [35]:
xuhui_kangjian.shape

(90, 7)

In [36]:
data_broken = data.iloc[data_broken.index, :]

In [37]:
data_broken.head()

,date,district,town_name,description,listed_price,attributes
9,20200208,xuhui,kangjian,A+好房！徐汇唯一一套户型正客厅大的双南两房！必看好房,460万,2室1厅 | 93.34平米 | 南 | 精装 | 28层 | 塔楼
250,20200208,pudong,nanmatou,房子2015年新加坡开发商，房子带产权车位，价格另算必看好房,1200万,3室2厅 | 142.71平米 | 南 | 精装 | 低楼层(共16层) | 板楼
326,20200208,pudong,nanmatou,中间楼层+经典一室半+视野开阔+产权干净无户口,185万,1室1厅 | 36.78平米 | 南 | 简装 | 中楼层(共6层) | 板楼
329,20200208,pudong,nanmatou,新加坡开发商，2015年次新房，高区两房朝南厅朝南,880万,2室2厅 | 104.6平米 | 南 | 精装 | 高楼层(共16层) | 板楼
440,20200208,pudong,nanmatou,三房两厅高楼层，户型方正，业主诚意出售到价签字,1140万,3室2厅 | 142.71平米 | 南 北 | 精装 | 高楼层(共16层) | 板楼


In [38]:
# 这些数据似乎只是少了建成年代
# 来看看attributes的长度分布
attributes_len = attributes.apply(lambda x: len(x)).value_counts()

In [39]:
attributes_len

7    15625
6      573
1        8
Name: attributes, dtype: int64

In [40]:
# 丢弃那8个只有一个attribute的行
data.drop(attributes[attributes.apply(lambda x: len(x))==1].index, inplace=True)

In [41]:
data.shape

(16198, 6)

In [42]:
#看剩下的有6个attributes的行是否都是缺建成年代
attributes[attributes.apply(lambda x: len(x))==6].str.contains('年', regex=False).any()

False

In [43]:
# 给缺建成年的补上0年
def insert_year(x, year=0):
    """insert list x with built year """
    assert len(x) == 6
    
    return x.insert(5, str(year)+'年建')
    
attributes[attributes.apply(lambda x: len(x))==6].apply(insert_year)

9        None
250      None
326      None
329      None
440      None
         ... 
15842    None
15898    None
15911    None
15916    None
15920    None
Name: attributes, Length: 573, dtype: object

In [44]:
# 检查一下
print(attributes[9])


['2室1厅 ', ' 93.34平米 ', ' 南 ', ' 精装 ', ' 28层  ', '0年建', ' 塔楼']


In [45]:
attributes.drop(attributes[attributes.apply(len)==1].index, inplace=True)

In [46]:
attributes.shape[0] == data.shape[0]


True

In [47]:
attributes_splited = pd.DataFrame(index=attributes.index)
attr_cols = ['layout', 'total_area', 'direction', 'decoration_rate', 'stairs', 'built_year', 'building_type' ]
for i, attribute in enumerate(attr_cols):
    print(attribute)
    attributes_splited[attribute] = attributes.apply(lambda x: x[i])

layout
total_area
direction
decoration_rate
stairs
built_year
building_type


In [48]:
attributes_splited.head(3)

,layout,total_area,direction,decoration_rate,stairs,built_year,building_type
0,2室0厅,53.92平米,南,精装,中楼层(共4层),1992年建,板楼
1,2室1厅,67.11平米,南,简装,6层,1990年建,板楼
2,2室1厅,68.03平米,南,精装,低楼层(共6层),1991年建,板楼


In [49]:
data = data.merge(attributes_splited, left_index=True, right_index=True)

In [50]:
data.shape

(16198, 13)

In [51]:
data.drop(['date', 'attributes'], axis=1, inplace=True)

In [52]:
data.head()

,district,town_name,description,listed_price,layout,total_area,direction,decoration_rate,stairs,built_year,building_type
0,xuhui,kangjian,中间楼层，欧式装修，全明边套净收眼底必看好房,375万,2室0厅,53.92平米,南,精装,中楼层(共4层),1992年建,板楼
1,xuhui,kangjian,花苑村紫竹园 2室1厅 468万必看好房,468万,2室1厅,67.11平米,南,简装,6层,1990年建,板楼
2,xuhui,kangjian,南北两房全明户型采光充足 近上海南站地铁站 诚售。必看好房,468万,2室1厅,68.03平米,南,精装,低楼层(共6层),1991年建,板楼
3,xuhui,kangjian,桂莘小区 双南双天井 精装三房 近地铁十二号线必看好房,520万,3室1厅,78.38平米,南,精装,低楼层(共6层),1997年建,板楼
4,xuhui,kangjian,精装电梯三房，近地铁，双阳台，闹中取静，值得拥有。必看好房,499万,3室1厅,110.66平米,东南,精装,28层,1996年建,塔楼


In [53]:
# 把价格，面积， 修建年转为数字型
listed_price = data['listed_price'].apply(lambda x: float(x.strip()[:-1]))
total_area = data['total_area'].apply(lambda x: float(x.strip()[:-2]))
built_year = data['built_year'].apply(lambda x: float(x.strip()[:-2]))
                                                  

ValueError: could not convert string to float: 

In [54]:
# 检查看built_year有何问题
data['built_year'].value_counts()

 1994年建     1160
 1995年建     1007
 2006年建      948
 2005年建      793
 2008年建      769
            ... 
 1941年建        1
 2019年建        1
 1956年建        1
 1945年建        1
 1973年建        1
Name: built_year, Length: 70, dtype: int64

In [55]:
built_year_format = data['built_year'].str.match(pat='^ *[0-9]+年建 *$')

In [56]:
built_year_format

0        True
1        True
2        True
3        True
4        True
         ... 
16201    True
16202    True
16203    True
16204    True
16205    True
Name: built_year, Length: 16198, dtype: bool

In [57]:
built_year_format.all()

False

In [58]:
data['built_year'][built_year_format==False]

9673     板楼 
Name: built_year, dtype: object

In [59]:
data.loc[9673, :]

district                      minhang
town_name                      maqiao
description        样板房装修！未住过！急售！临天然大河
listed_price                    1450万
layout                          3室3厅 
total_area                  299.48平米 
direction                          北 
decoration_rate                   精装 
stairs                           2层  
built_year                        板楼 
building_type                    独栋别墅
Name: 9673, dtype: object

In [60]:
# 只有一个数据有问题，修复它
data.loc[9673, 'built_year'] = '0年建'

In [61]:
built_year = data['built_year'].apply(lambda x: int(x.strip()[:-2]))

In [62]:
data['listed_price'] = listed_price
data['total_area'] = total_area
data['built_year'] = built_year

In [63]:
data.head()

,district,town_name,description,listed_price,layout,total_area,direction,decoration_rate,stairs,built_year,building_type
0,xuhui,kangjian,中间楼层，欧式装修，全明边套净收眼底必看好房,375.0,2室0厅,53.92,南,精装,中楼层(共4层),1992,板楼
1,xuhui,kangjian,花苑村紫竹园 2室1厅 468万必看好房,468.0,2室1厅,67.11,南,简装,6层,1990,板楼
2,xuhui,kangjian,南北两房全明户型采光充足 近上海南站地铁站 诚售。必看好房,468.0,2室1厅,68.03,南,精装,低楼层(共6层),1991,板楼
3,xuhui,kangjian,桂莘小区 双南双天井 精装三房 近地铁十二号线必看好房,520.0,3室1厅,78.38,南,精装,低楼层(共6层),1997,板楼
4,xuhui,kangjian,精装电梯三房，近地铁，双阳台，闹中取静，值得拥有。必看好房,499.0,3室1厅,110.66,东南,精装,28层,1996,塔楼


In [65]:
data.dtypes

district            object
town_name           object
description         object
listed_price       float64
layout              object
total_area         float64
direction           object
decoration_rate     object
stairs              object
built_year           int64
building_type       object
dtype: object

In [66]:
# 把整理好的数据保存
data.to_csv(base_path+'reorganized_data/reorganized.csv', index=False)